In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import joblib

# Cargar modelo y transformadores
modelo = joblib.load("modelo_rf_output/modelo_rf.pkl")
scaler = joblib.load("modelo_rf_output/scaler.pkl")
label_encoder = joblib.load("modelo_rf_output/label_encoder.pkl")

# Inicializar MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Función para extraer landmarks (sin visibilidad)
def extraer_landmarks(results):
    if not results.pose_landmarks:
        return None
    return [v for lm in results.pose_landmarks.landmark for v in (lm.x, lm.y, lm.z)]

# Iniciar captura desde cámara
cap = cv2.VideoCapture(1)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    # Dibujar los landmarks en el frame
    mp.solutions.drawing_utils.draw_landmarks(
        frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Extraer y predecir
    landmarks = extraer_landmarks(results)
    if landmarks:
        X = np.array(landmarks).reshape(1, -1)
        X_scaled = scaler.transform(X)
        y_pred = modelo.predict(X_scaled)
        actividad = label_encoder.inverse_transform(y_pred)[0]

        # Mostrar actividad en pantalla
        cv2.putText(frame, f"Actividad: {actividad}", (10, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)

    cv2.imshow("Clasificador de Actividades (en vivo)", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1747590022.429308   18126 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M2
OpenCV: out device of bound (0-0): 1
OpenCV: camera failed to properly initialize!


W0000 00:00:1747590022.501192   33882 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747590022.511682   33882 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


: 